In [1]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
def generate_basis(N_sites, N_particles): # Checked OK
    """Generate all Fock basis states with N_particles electrons."""
    basis = []
    for comb in combinations(range(N_sites), N_particles):
        state = [0] * N_sites
        for idx in comb:
            state[idx] = 1
        basis.append(tuple(state))
    return basis

In [ ]:
def fock_state_to_index(basis): # Checked OK
    """Map each Fock state to its index in the basis."""
    
    return {state: i for i, state in enumerate(basis)}

In [ ]:
def apply_annihilation(state, i): # Checked OK
    if state[i] == 0:
        return None, 0
    new_state = list(state)
    new_state[i] = 0
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

def apply_creation(state, i): # Checked OK
    if state[i] == 1:
        return None, 0
    new_state = list(state)
    new_state[i] = 1
    sign = 1
    for j in range(i):
        if state[j] == 1:
            sign *= -1
    return tuple(new_state), sign

In [30]:
def build_Hamiltonian(basis, N_unit_cells, v, w):
    dim = len(basis)
    H = csr_matrix((dim, dim), dtype=np.float64)
    state_to_idx = fock_state_to_index(basis)

    for idx, state in enumerate(basis):
        # Intra-cell hopping: A <-> B
        for i in range(N_unit_cells):
            site_A = 2*i
            site_B = 2*i + 1

            # B -> A
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

            # A -> B (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += v * sign_up * sign_down

        # Inter-cell hopping: A_{i+1} <-> B_i
        for i in range(N_unit_cells - 1):
            site_B = 2*i + 1
            site_A_next = 2*(i+1)

            # B_i -> A_{i+1}
            down_state, sign_down = apply_annihilation(state, site_B)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_A_next)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

            # A_{i+1} -> B_i (h.c.)
            down_state, sign_down = apply_annihilation(state, site_A_next)
            if down_state is not None:
                up_state, sign_up = apply_creation(down_state, site_B)
                if up_state in state_to_idx:
                    jdx = state_to_idx[up_state]
                    H[jdx, idx] += w * sign_up * sign_down

    return H

In [51]:
N_unit_cells = 4
v = 1
w = 2
N_orbitals = 2 * N_unit_cells
N_particles = N_unit_cells

In [52]:
basis = generate_basis(N_orbitals, N_particles)
print(len(basis))

70


In [15]:
state_to_idx = fock_state_to_index(basis)
print(state_to_idx)

{(1, 1, 0, 0): 0, (1, 0, 1, 0): 1, (1, 0, 0, 1): 2, (0, 1, 1, 0): 3, (0, 1, 0, 1): 4, (0, 0, 1, 1): 5}


In [19]:
print(apply_creation(basis[0], 2))

((1, 1, 1, 0), 1)


In [53]:
H = build_Hamiltonian(basis, N_unit_cells, v, w)
print(H)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 280 stored elements and shape (70, 70)>
  Coords	Values
  (0, 1)	2.0
  (1, 0)	2.0
  (1, 2)	1.0
  (1, 5)	1.0
  (2, 1)	1.0
  (2, 3)	2.0
  (2, 6)	1.0
  (3, 2)	2.0
  (3, 4)	1.0
  (3, 7)	1.0
  (4, 3)	1.0
  (4, 8)	1.0
  (5, 1)	1.0
  (5, 6)	1.0
  (5, 15)	2.0
  (6, 2)	1.0
  (6, 5)	1.0
  (6, 7)	2.0
  (6, 9)	2.0
  (6, 16)	2.0
  (7, 3)	1.0
  (7, 6)	2.0
  (7, 8)	1.0
  (7, 10)	2.0
  (7, 17)	2.0
  :	:
  (62, 52)	2.0
  (62, 59)	2.0
  (62, 61)	1.0
  (62, 63)	2.0
  (62, 66)	1.0
  (63, 53)	2.0
  (63, 60)	2.0
  (63, 62)	2.0
  (63, 64)	1.0
  (63, 67)	1.0
  (64, 54)	2.0
  (64, 63)	1.0
  (64, 68)	1.0
  (65, 61)	1.0
  (65, 66)	1.0
  (66, 62)	1.0
  (66, 65)	1.0
  (66, 67)	2.0
  (67, 63)	1.0
  (67, 66)	2.0
  (67, 68)	1.0
  (68, 64)	1.0
  (68, 67)	1.0
  (68, 69)	2.0
  (69, 68)	2.0


c:\Users\DHIMAN\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


In [54]:
eigenvalues, eigenvectors = np.linalg.eigh(H.toarray())

In [55]:
print(eigenvalues)

[-6.85406818e+00 -6.66449300e+00 -5.15908490e+00 -5.15908490e+00
 -4.42703409e+00 -4.42703409e+00 -3.93244219e+00 -3.93244219e+00
 -3.65367679e+00 -3.46410162e+00 -2.92162598e+00 -2.92162598e+00
 -2.73205081e+00 -2.73205081e+00 -2.42703409e+00 -2.42703409e+00
 -2.23745891e+00 -2.23745891e+00 -2.18957518e+00 -2.00000000e+00
 -1.69498328e+00 -1.69498328e+00 -1.50540811e+00 -1.50540811e+00
 -1.22664270e+00 -1.22664270e+00 -1.20039139e+00 -1.01081621e+00
 -7.32050808e-01 -7.32050808e-01 -4.94591894e-01 -4.94591894e-01
 -1.80361341e-15 -1.03275891e-15 -4.74344379e-16 -3.65052215e-16
  1.77028226e-15  3.22454111e-15  4.94591894e-01  4.94591894e-01
  7.32050808e-01  7.32050808e-01  1.01081621e+00  1.20039139e+00
  1.22664270e+00  1.22664270e+00  1.50540811e+00  1.50540811e+00
  1.69498328e+00  1.69498328e+00  2.00000000e+00  2.18957518e+00
  2.23745891e+00  2.23745891e+00  2.42703409e+00  2.42703409e+00
  2.73205081e+00  2.73205081e+00  2.92162598e+00  2.92162598e+00
  3.46410162e+00  3.65367

In [56]:
wf_ps = np.zeros((len(basis), len(basis[0])))

basis = np.array(basis)
for j, state in enumerate(eigenvectors):
    for i, weight in enumerate(state):
        wf_ps[j] += weight * basis[i]

In [ ]:
sites_arr = np.arange(N_unit_cells*2)
for i, wf in enumerate(wf_ps):
    # plt.plot(sites_arr, wf, label=f'{i}')
    plt.plot(sites_arr, wf**2, label=f'{i}')
    plt.legend()
    plt.grid()
    plt.show()
